In [7]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
def CreateMNISTDataset(dir):
    df = pd.read_csv(dir)
    Xdf = df.iloc[:, 1:].to_numpy()
    ydf = df.iloc[:, 0].to_numpy()
    return Xdf, ydf

In [4]:
train_cwd = os.getcwd() + '/mnist_train.csv'
test_cwd = os.getcwd() + '/mnist_test.csv'

X_train, y_train = CreateMNISTDataset(train_cwd)
X_test, y_test = CreateMNISTDataset(test_cwd)

X_ttrain, y_ttrain = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)
X_ttest, y_ttest = tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)

In [5]:
input_dim = X_train.shape[1]
num_labels = 10

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(10, activation = 'softmax'))


In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

opt = tf.keras.optimizers.Adam(learning_rate= 0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
print(model.summary())

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
n_epochs = 10
batch_size = 8

In [ ]:
hist = model.fit(X_ttrain, y_ttrain,
                 batch_size=batch_size,
                 epochs=n_epochs,
                 verbose=2,
                 validation_split=0.2)